In [3]:
import pandas as pd
import re
import hanja
from tqdm import tqdm
from konlpy.tag import Okt
from pykospacing import Spacing
from collections import Counter

In [4]:
ys_youtube = pd.read_csv('strings_Yjoined.csv')

In [6]:
ys_youtube = ys_youtube.shift(1)
ys_youtube.columns = ['comment']
ys_youtube.fillna("예산 텃새가 웃기는 심하고 동네예요", inplace=True)

In [5]:
ys_youtube

,예산 텃새가 웃기는 심하고 동네예요
0,자본주의 사회주의 시장경제 국가도 대한민국이
1,장사하는사람들이백대표가다해놓어니뒤통수치네백대표님 저런사람들도와주지마새요
2,백씨너무싫다
3,예산오지마라 진짜예산에서 꼴보기싫으니카 정도껏 백종원
4,심해요 흉성은
...,...
31658,일주일에 2편 미쳤다 구스타 작업량
31659,많관부입니다
31660,감사합니다 봐주셔서
31661,감사 좋은 공유


In [7]:
comment_list = []
for i in range(len(ys_youtube)):
    comment_list.append(ys_youtube['comment'].iloc[i])
    
ys_comment_result = pd.DataFrame(comment_list, columns=["comment"])

In [8]:
ys_comment_result

,comment
0,예산 텃새가 웃기는 심하고 동네예요
1,자본주의 사회주의 시장경제 국가도 대한민국이
2,장사하는사람들이백대표가다해놓어니뒤통수치네백대표님 저런사람들도와주지마새요
3,백씨너무싫다
4,예산오지마라 진짜예산에서 꼴보기싫으니카 정도껏 백종원
...,...
31658,미칫다이 구스타
31659,일주일에 2편 미쳤다 구스타 작업량
31660,많관부입니다
31661,감사합니다 봐주셔서


In [9]:
import re
# 한글만 남기기
def extract_word(lst):
    hangul = re.compile('[^가-힣]') 
    return [hangul.sub('', text) for text in lst]

cleaned_comment_result = extract_word(comment_list)

In [10]:
from pykospacing import Spacing
spacing = Spacing()
# 띄어쓰기 교정
def apply_korean_spacing(text):
    return spacing(text)

cleaned_comment_result = [apply_korean_spacing(text) for text in cleaned_comment_result]

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Hugging Face 모델 로딩
model_name = "WhitePeak/bert-base-cased-Korean-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    
    sentiment_label = ["Negative", "Positive"][predicted_class_id]
    
    return sentiment_label

# inference 수행
sentiment_results = [get_sentiment(text) for text in cleaned_comment_result]

# 결과 출력
for text, sentiment in zip(cleaned_comment_result, sentiment_results):
    print(f"Text: {text}\nSentiment: {sentiment}\n")


Text: 예산 텃새가 웃기는 심하고 동네예요
Sentiment: Negative

Text: 자본주의 사회주의 시장경제 국가도 대한민국이
Sentiment: Negative

Text: 장사하는 사람들이 백 대표가 다 해놓어 니 뒤통수 치네 백 대표님 저런 사람들도와주지마 새요
Sentiment: Negative

Text: 백씨 너무 싫다
Sentiment: Negative

Text: 예산 오지 마라진짜 예산에서 꼴 보기 싫으니 카 정도껏 백종원
Sentiment: Positive

Text: 심해요 흉성은
Sentiment: Negative

Text: 좋아져는데 왜 그려지 건물주가 깨끗하고 예산
Sentiment: Negative

Text: 익사해라 안고 점포
Sentiment: Negative

Text: 시장 사람들 예산시는 시장 버리고 아무도
Sentiment: Negative

Text: 건강도 챙기세 요응원합니다
Sentiment: Positive

Text: 갑질도 천벌생계를 일해서 상인들이
Sentiment: Negative

Text: 멍청하면 족발까지 하셨기에 그 기회를 울엄마가 포장마차
Sentiment: Negative

Text: 건물 건물주들이 기적이고 세입자 필요가 없어
Sentiment: Negative

Text: 충청도는 충청도 인가요 그래 유예산이
Sentiment: Negative

Text: 손님이 안 가야겠네요
Sentiment: Negative

Text: 잘 안 믿는데 백종원 씨는 믿습니다 저는 사람을
Sentiment: Negative

Text: 지랄들 안 되도 떨긴 하지 싶으면 지랄들을
Sentiment: Negative

Text: 건강하세요 힘내세 요 응원합니다입니다 존경합니다
Sentiment: Positive

Text: 사람들 안 가게 하자 건물주가 오픈해서 간단하게 퇴거
Sentiment: Negative

Text: 씨발 너무 하네 생각해야지 없던 시절을 해도
Sentiment: Negati

In [12]:
sentiment_results

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',

In [13]:
ys_comment_result['cleaned_comment'] = cleaned_comment_result
ys_comment_result['sentiment'] = sentiment_results

In [14]:
ys_comment_result.head(40)

,comment,cleaned_comment,sentiment
0,예산 텃새가 웃기는 심하고 동네예요,예산 텃새가 웃기는 심하고 동네예요,Negative
1,자본주의 사회주의 시장경제 국가도 대한민국이,자본주의 사회주의 시장경제 국가도 대한민국이,Negative
2,장사하는사람들이백대표가다해놓어니뒤통수치네백대표님 저런사람들도와주지마새요,장사하는 사람들이 백 대표가 다 해놓어 니 뒤통수 치네 백 대표님 저런 사람들도와주...,Negative
3,백씨너무싫다,백씨 너무 싫다,Negative
4,예산오지마라 진짜예산에서 꼴보기싫으니카 정도껏 백종원,예산 오지 마라진짜 예산에서 꼴 보기 싫으니 카 정도껏 백종원,Positive
5,심해요 흉성은,심해요 흉성은,Negative
6,좋아져는데 왜그려지 건물주가 깨끗하고 예산,좋아져는데 왜 그려지 건물주가 깨끗하고 예산,Negative
7,익사해라 안고 점포,익사해라 안고 점포,Negative
8,시장사람들 예산시는 시장 버리고 아무도,시장 사람들 예산시는 시장 버리고 아무도,Negative
9,건강도 챙기세요 응원합니다,건강도 챙기세 요응원합니다,Positive


In [17]:
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 동사, 명사, 형용사만 추출하는 함수 정의
def extract_keywords(text):
    tokens = okt.pos(text)
    keywords = [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]
    return keywords  # 카운팅을 위해 리스트로 반환

# cleaned_comment 열에서 키워드 추출 후 새로운 열 생성
ys_comment_result['keywords'] = ys_comment_result['cleaned_comment'].apply(extract_keywords)

In [15]:
ys_comment_result['sentiment'].value_counts()

sentiment
Negative    19208
Positive    12455
Name: count, dtype: int64

In [19]:
# Positive, Negative 그룹으로 나누기
positive_comments = ys_comment_result[ys_comment_result['sentiment'] == 'Positive']['keywords']
negative_comments = ys_comment_result[ys_comment_result['sentiment'] == 'Negative']['keywords']

# 리스트를 평탄화 (각 리스트를 하나의 리스트로 만들어 키워드를 모두 모으기)
positive_keywords = [word for keywords in positive_comments for word in keywords]
negative_keywords = [word for keywords in negative_comments for word in keywords]

# 각 감정 그룹별로 키워드 카운트
positive_keyword_counts = Counter(positive_keywords)
negative_keyword_counts = Counter(negative_keywords)

# DataFrame으로 변환하여 보기 좋게 정리
positive_df = pd.DataFrame(positive_keyword_counts.items(), columns=['Keyword', 'Count']).sort_values(by='Count', ascending=False)
negative_df = pd.DataFrame(negative_keyword_counts.items(), columns=['Keyword', 'Count']).sort_values(by='Count', ascending=False)


In [21]:
positive_df.head(10)

,Keyword,Count
0,예산,979
460,축제,888
185,맥주,866
10,백종원,815
135,감사합니다,791
39,대표,773
404,술,655
85,백,609
17,요,554
14,합니다,431


In [23]:
negative_df.head(50)

,Keyword,Count
0,예산,1680
128,축제,1136
67,백종원,1007
12,사람,972
63,안,758
36,시장,721
232,거,680
29,건물,612
686,맥주,548
885,바베큐,532


In [24]:
positive_df.to_csv('예산유튜브_긍정단어.csv', encoding = 'cp949')
negative_df.to_csv('예산유튜브_부정단어.csv', encoding = 'cp949')